In [3]:
import pandas as pd
import re
import pdfplumber

def split_rows(row):
    # Split the row by '\n' if any element contains '\n'
    if any(isinstance(val, str) and '\n' in val for val in row):
        split_values = [str(val).split('\n') if isinstance(val, str) else [val] for val in row]
        # Determine the maximum length of sublists in the original data
        max_length = max(len(sublist) for sublist in split_values)
        # Fill shorter sublists with empty strings to match the maximum length
        filled_data = [sublist + [''] * (max_length - len(sublist)) for sublist in split_values]
        # Use zip to transpose the list and then convert it to a list of lists
        restructured_data = [list(row) for row in zip(*filled_data)]
        return restructured_data
    else:
        return [list(row)]

def correct_df_list(df_list, rows):
    corrected_df_list = []
    i = 0
    while i < len(df_list):
        current_row = df_list[i]
        
        # Check if the next row exists and if it should be merged
        if i + 1 < len(df_list) and pd.isna(df_list[i+1].iloc[0, 0]):
            next_row = df_list[i+1]
            
            # Combine the two rows
            combined_row = pd.concat([current_row.iloc[:, :2], next_row.iloc[:, 2:]], axis=1)
            
            # Find the corresponding row in the extracted text
            date = current_row.iloc[0, 1]
            matched_row = next((row for row in rows if row.startswith(date)), None)
            
            if matched_row:
                # Split the matched row and update the combined row
                parts = re.split(r'\s{2,}', matched_row.strip())
                if len(parts) >= 4:
                    combined_row.iloc[0, 0] = parts[0]  # Date
                    combined_row.iloc[0, 1] = ' '.join(parts[1:-2])  # Description
                    combined_row.iloc[0, 2] = parts[-2]  # Deposit/Withdrawal
                    combined_row.iloc[0, 3] = parts[-1]  # Balance
            
            corrected_df_list.append(combined_row)
            i += 2
        else:
            corrected_df_list.append(current_row)
            i += 1
    
    return pd.concat(corrected_df_list, ignore_index=True)

file_path = 'C:/Users/User/Downloads/Merged stmt (1).pdf'

pdf = pdfplumber.open(file_path)

df_list = []  # Use a list to store DataFrames

for i in range(len(pdf.pages)):
    page = pdf.pages[i]
    table = page.extract_table(table_settings={"horizontal_strategy": "text"})

    if table:
        table_df = pd.DataFrame(table[2:])
        if len(table_df.columns) == 5:
            for _, row in table_df.iterrows():
                split_row_list = split_rows(row)
                for split_row in split_row_list:
                    df_list.append(pd.DataFrame([split_row], columns=table_df.columns))
page_num = 0
with pdfplumber.open(file_path) as pdf:
    bal = []
    text = ""
    for page in pdf.pages:
        page_num += 1
        text = f'{text} \n{page.extract_text()}'
rows = text.split('\n')
# Usage
corrected_df = correct_df_list(df_list, rows)

In [59]:
def check_and_fill(df_list):
    df = pd.concat(df_list, ignore_index=True)
    AMOUNT_REGEX = r'\.\d{2}'
    
    for i in range(1, len(df) - 1):
        try:
            # Check if first column is empty string and any of columns 2, 3, 4 match the regex
            if df.iloc[i, 0] == "" and any(isinstance(df.iloc[i, col], str) and 
                                           re.search(AMOUNT_REGEX, str(df.iloc[i, col])[-3:]) 
                                           for col in [2, 3, 4]):
                
                matching_columns = [col for col in [2, 3, 4] 
                                    if isinstance(df.iloc[i, col], str) and 
                                    re.search(AMOUNT_REGEX, str(df.iloc[i, col])[-3:])]
                
                if matching_columns:
                    print(f"Row {i}: Matching columns: {matching_columns}")
                    print(f"Values: {[df.iloc[i, col] for col in matching_columns]}")
                    
                    prev_amt_check = all(pd.isna(df.iloc[i-1, j]) or df.iloc[i-1, j] == "" for j in matching_columns)
                    next_amt_check = all(pd.isna(df.iloc[i+1, j]) or df.iloc[i+1, j] == "" for j in matching_columns)
                    prev_date_check = all(df.iloc[i-1, j] != "" for j in [0,1] if pd.notna(df.iloc[i-1, j]))
                    next_date_check = all(df.iloc[i+1, j] != "" for j in [0,1] if pd.notna(df.iloc[i+1, j]))
                    
                    if next_amt_check and next_date_check:
                        for col in matching_columns:
                            df.iloc[i+1, col] = df.iloc[i, col]
                            df.iloc[i, col] = ""
                    elif prev_amt_check and prev_date_check:
                        for col in matching_columns:
                            df.iloc[i-1, col] = df.iloc[i, col]
                            df.iloc[i, col] = ""
        
        except Exception as e:
            print(f"Error processing row {i}: {str(e)}")
            print(f"Row data: {df.iloc[i]}")
    
    return df

In [60]:
file_path = 'C:/Users/User/Downloads/Merged stmt (1).pdf'

pdf = pdfplumber.open(file_path)

df_list = []  # Use a list to store DataFrames

for i in range(len(pdf.pages)):
    page = pdf.pages[i]
    table = page.extract_table(table_settings={"horizontal_strategy": "text"})

    if table:
        table_df = pd.DataFrame(table[2:])
        if len(table_df.columns) == 5:
            for _, row in table_df.iterrows():
                split_row_list = split_rows(row)
                for split_row in split_row_list:
                    df_list.append(pd.DataFrame([split_row], columns=table_df.columns))
# Usage
corrected_df = check_and_fill(df_list)

Row 1: Matching columns: [4]
Values: ['7,609.38']
Row 24: Matching columns: [2, 4]
Values: ['1,220.00', '8,340.88']
Row 52: Matching columns: [4]
Values: ['6,910.38']
Row 117: Matching columns: [2]
Values: ['1,350.00']
Row 147: Matching columns: [4]
Values: ['12,965.88']


In [61]:
corrected_df

,0,1,2,3,4
0,,,,,
1,,Balance from previous statement,,,"7,609.38"
2,,,,,
3,30-03-2024,Instant Transfer at KLM,"1,195.00",,"8,804.38"
4,,Cake Payment,,,
...,...,...,...,...,...
246,,20240625CIBBMYKL010ORM46906467,,,
247,,None,None,None,None
248,Total Withdrawals / Jumlah Pengeluaran : 8 3 6...,None,None,None,None
249,,None,None,None,None


In [62]:
corrected_df.to_csv("test.csv")

In [1]:
from pdfminer.high_level import extract_pages
import os
from pdfminer.layout import LTTextContainer, LTChar
from python.font_check import text_extraction, process_fonts, extract_fonts, draw_rectangles

file_path = "C:/Users/User/Downloads/BL UOB STATEMENT.pdf"


results = []
for pagenum, page in enumerate(extract_pages(file_path)):

    # Iterate the elements that composed a page
    for element in page:

        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            result = text_extraction(element)
            results.append((pagenum, result))

font_data = extract_fonts(file_path)

print(font_data)

[{'/F00': 'Helvetica', '/F1': 'Arial', '/V10F': 'Helvetica', '/V11F': 'Helvetica', '/V12F': 'Helvetica-Oblique', '/V13F': 'Helvetica-Bold', '/V14F': 'Helvetica-BoldOblique', '/V15F': 'Helvetica-Bold', '/V16F': 'Helvetica-Oblique', '/V17F': 'Helvetica-Oblique', '/V18F': 'Helvetica', '/V19F': 'Helvetica-Oblique', '/V20F': 'Helvetica', '/V21F': 'Helvetica-Bold', '/V8F': 'Helvetica', '/V9F': 'Helvetica'}, {'/F00': 'Helvetica', '/F1': 'Arial', '/V10F': 'Helvetica', '/V11F': 'Helvetica', '/V12F': 'Helvetica-Oblique', '/V13F': 'Helvetica-Bold', '/V15F': 'Helvetica-Bold', '/V16F': 'Helvetica-Oblique', '/V17F': 'Helvetica-Oblique', '/V18F': 'Helvetica', '/V8F': 'Helvetica', '/V9F': 'Helvetica'}, {'/F00': 'Helvetica', '/F1': 'Arial', '/V10F': 'Helvetica', '/V11F': 'Helvetica', '/V12F': 'Helvetica-Oblique', '/V13F': 'Helvetica-Bold', '/V15F': 'Helvetica-Bold', '/V16F': 'Helvetica-Oblique', '/V17F': 'Helvetica-Oblique', '/V18F': 'Helvetica', '/V8F': 'Helvetica', '/V9F': 'Helvetica'}, {'/F00': 'Hel